# 学习pipeline内部原理


In [1]:
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoModelForSequenceClassification


d:\Anaconda\envs\d2lc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### pipeline过程

Tokenizer ==> Model ==>  Post Processing

### PipeLine流程拆分

Tokenizer

```
将输入拆分为单词、子单词或符号（如标点符号），称为 token（标记）
将每个标记（token）映射到一个数字，称为 input ID（inputs ID）
添加模型需要的其他输入，例如特殊标记（如 [CLS] 和 [SEP] ）
位置编码：指示每个标记在句子中的位置。
段落标记：区分不同段落的文本。
特殊标记：例如 [CLS] 和 [SEP] 标记，用于标识句子的开头和结尾。
```

In [2]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"   # 这里是一个分类模型
# 下载该预训练模型的tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [3]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I like this so much"
]
inputs = tokenizer(raw_inputs,padding=True,truncation=True,return_tensors='pt')
print(inputs) # 可以看到单词被转为了对应编码id

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  2066,  2023,  2061,  2172,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [4]:
# 下载该预训练模型的模型部分(只包含基本transformer块，不含tokenizer和post processing)
# post processing 中包含了一个ffn
model = AutoModel.from_pretrained(checkpoint)

In [5]:
outputs = model(**inputs) # **是将inputs解包成关键字形式输入
print(outputs)
# 输入的是tokenizer后的序列id，输出logits 格式（batchsize，sequence length，hidden size）

BaseModelOutput(last_hidden_state=tensor([[[-0.1798,  0.2333,  0.6321,  ..., -0.3017,  0.5008,  0.1481],
         [ 0.2758,  0.6497,  0.3200,  ..., -0.0760,  0.5136,  0.1329],
         [ 0.9046,  0.0985,  0.2950,  ...,  0.3352, -0.1407, -0.6464],
         ...,
         [ 0.1466,  0.5661,  0.3235,  ..., -0.3376,  0.5100, -0.0561],
         [ 0.7500,  0.0487,  0.1738,  ...,  0.4684,  0.0030, -0.6084],
         [ 0.0519,  0.3729,  0.5223,  ...,  0.3584,  0.6500, -0.3883]],

        [[ 0.4208,  0.1957,  0.1640,  ...,  0.5211,  0.9966, -0.4569],
         [ 0.6533,  0.3599,  0.1282,  ...,  0.4988,  1.0136, -0.3346],
         [ 0.6274,  0.3405,  0.3473,  ...,  0.4319,  0.9345, -0.3468],
         ...,
         [ 0.2336,  0.1154,  0.1394,  ...,  0.5895,  0.8571, -0.4210],
         [ 0.2652,  0.1805,  0.1310,  ...,  0.5579,  0.8555, -0.4207],
         [ 0.2358,  0.1306,  0.1516,  ...,  0.6039,  0.8488, -0.4314]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


In [6]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)  # 下载带分类头（也就是模型最后的ffn层输出分类结果）的模型
outputs = model(**inputs)
print(outputs)
# 相比上面那个不带分类头的模型，这次输出的不再是隐变量，而是分类结果，只不过还没做softmax，id转label等操作

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [-4.2844,  4.6212]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


所有Transformers 模型的输出都是 logits，因为训练时的损失函数通常会将最后的激活函数（如 SoftMax）与实际的损失函数（如交叉熵）融合）

In [7]:
import torch
# 做softmax
predictions = torch.nn.functional.softmax(outputs.logits,dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [1.3561e-04, 9.9986e-01]], grad_fn=<SoftmaxBackward0>)


In [8]:
# 接下来做 id转label，得先知道id和label的映射关系，需要查看我们刚刚使用的模型的id转换配置
# 输出结果的单个样本的下标0，1，。。。对应id
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

### 详解三个流程中的Transformer模型部分

In [9]:
from transformers import BertConfig,BertModel

config = BertConfig()       # 库中默认的bert配置
model = BertModel(config)   #使用默认配置创建模型会使用随机值对其进行初始化：

In [10]:
print(config)

BertConfig {
  "_attn_implementation_autoset": true,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [11]:
# 刚刚创建的模型，需要从头开始训练
# 直接加载预训练模型
model = BertModel.from_pretrained("bert-base-cased") # 加载google的模型参数

# 相比直接使用 AutoModel.from_pretained("bert-base-cased") 或者 pipeline("bert-base-cased")
# 这种方式更方便阅读

In [12]:
# 保存模型 两个文件，架构配置和模型参数
model.save_pretrained("./my_model")

In [13]:
# 我们刚刚使用了上面的tokenizer来将句子转为词汇表索引
# 因为transformer库的模型只接受词汇表索引
# 我们手动构建一个输入来测测
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

# *********** 注意 ： 这里必须是句子列表，如果endoced_sequences里面只有一个句子，也要[[]]

# 转为张量，transformers库的模型只接受张量
import torch
model_inputs = torch.tensor(encoded_sequences)

# 模型接受并给出隐状态结果
output = model(model_inputs)
print(output.last_hidden_state.shape)

torch.Size([3, 4, 768])


### 详解三个流程中的Tokenizer模型部分 （英语）

##### 1. 基于单词的tokenization (word-based tokenization)

> 目标是将原始文本拆分为单词并为每个单词找到一个数字表示

In [14]:
tokenized_text = "Jim Henson was a puppeteer".split()  # 直接调用split函数切割出来单词
print(tokenized_text)

# 我们最终可以得到一些非常大的“词汇表（vocabulary）”，其中词汇表的大小由我们在语料库中拥有的独立 tokens 的总数确定。
# 每个单词都分配了一个 ID，从 0 开始一直到词汇表的大小。模型使用这些 ID 来识别每个词。

# 我们需要一个自定义 token 来表示不在我们词汇表中的单词。这被称为“unknown” token，通常表示为“[UNK]”或“<unk>”

['Jim', 'Henson', 'was', 'a', 'puppeteer']


##### 2. 基于字符的tokenization (character-based tokenization)

> 使用26个字母+标点+特殊符号

这样做会导致我们的模型需要处理大量的 tokens：虽然一个单词在基于单词的 tokenizer 中只是一个 token，但当它被转换为字符时，很可能就变成了 10 个或更多的 tokens

为了两全其美，我们可以使用结合这两种方法的第三种技术：基于子词（subword）的 tokenization。

##### 3. 基于子词（subword）的 tokenization

基于子词（subword）的 tokenization 算法依赖于这样一个原则：常用词不应被分解为更小的子词，但罕见词应被分解为有意义的子词。

例如，“annoyingly”可能被视为一个罕见的词，可以分解为“annoying”和“ly”。这两者都可能作为独立的子词并且出现得更频繁，同时“annoyingly”的含义通过“annoying”和“ly”的复合含义得以保留。


### 如果是中文怎么分词呢？

NJU的统计模型课程中有，可以检查老师给的ppt

基于词典的分词算法，也叫字符串匹配分词算法，将待分词的字符串与已构建好的词典中的词进行匹配，若匹配成功则识别出该词。

正向最大匹配法

逆向最大匹配法

双向匹配分词法

基于统计的机器学习算法

基于理解的分词方法


### tokenizer加载和保存

In [15]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
# 也可以AutoTokenizer.from_pretrained


# 保存
tokenizer.save_pretrained("./my_tokenizer")

('./my_tokenizer\\tokenizer_config.json',
 './my_tokenizer\\special_tokens_map.json',
 './my_tokenizer\\vocab.txt',
 './my_tokenizer\\added_tokens.json')

In [16]:
print(tokenizer("Using a Transformer network is simple"))


{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [17]:
tokens = tokenizer.tokenize("Using a Transformer network is simple")
print(tokens)

ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']
[7993, 170, 13809, 23763, 2443, 1110, 3014]


### tokenizer id转回label

In [18]:
tokenizer.decode([7993, 170, 13809, 23763, 2443, 1110, 3014])

'Using a Transformer network is simple'

### 批量输入示例

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

# 输入多个batch   但是如果两个句子的长度不一致，需要 pad
input_ids = torch.tensor([ids,ids])
output = model(input_ids)
print("Logits:", output.logits)



Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)
Logits: tensor([[-2.7276,  2.8789],
        [-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [ ]:
# 手动使用 tokenizer.pad_token_id  会发现第二个句子的两次model输出值不一样，说明模型将pad值也纳入了考虑
sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


In [34]:
# 想要模型无视pad值，设置mask
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]
attention_mask = [
    [1,1,1],
    [1,1,0]
]
print(model(torch.tensor(batched_ids),attention_mask = torch.tensor(attention_mask)))

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [37]:
# tokenizer一次可以完成所有以上操作 mask,张量,inputs
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]
tokenizer(sequence)

# 可以指定句子长度
tokenizer(sequence,padding="longest") # 填充到最长句子长度
tokenizer(sequence,padding="max_length") # 填充到模型最大长度
tokenizer(sequence,padding="max_length",max_length=8) # 可以手动指定最大长度
# 可以截断句子
tokenizer(sequence,max_length=8,truncation=True)

# 返回 PyTorch tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")

# 返回 TensorFlow tensors
# model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

# 返回 NumPy arrays
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")

In [40]:
# 如果使用tokenizer()的方式，会自动添加特殊字符

sequence = "I've been waiting for a HuggingFace course my whole life."
print(tokenizer.tokenize(sequence))

inputs = tokenizer(sequence)
tokenizer.decode(inputs["input_ids"])

['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.']


"[CLS] i've been waiting for a huggingface course my whole life. [SEP]"